[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pytextbook/pytextbook/blob/main/7.1.%20seoul-120-LDA.ipynb)

## 토픽 모델링
* 실습을 위해 pyLDAvis 설치
* colab사용시 설치 후에도 제대로 동작하지 않거나 오류가 나면 런타임 재실행!

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# pyLDAvis 는 scikit-learn 의존성 문제로 해당 실습은 아래 버전으로 진행을 추천
!pip install -q scikit-learn==1.0.0
!pip install -U -q pyLDAvis==3.4.0

     |████████████████████████████████| 1.7 MB 7.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done


In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 라이브러리 로드

In [ ]:
# 필요 라이브러리를 로드
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [5]:
df = pd.read_csv("/content/drive/MyDrive/claw/filter_민원분류.csv")
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(3081, 6)

In [6]:
df.head(3)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Name,Value.age,Value.sex,Value.dep_name,UpdatedContent,UpdatedTitle
0,list,NONE,NONE,건설안전과,경기도 부천시 오정구 오정동 ***-*\n안녕하세요 \n여기는. 오정동 우체국 옆 ...,안녕하세요 여기는. 오정동 우체국 옆 상가 주차장 입
1,list,NONE,NONE,도시미관과,( https://www.safetyreport.go.kr/fileDown/singo/,쓰레기기를 낮애버려놓아 더니기불편해요* 안전신문고
2,list,30,1,건설안전과,부천 원미구 길주로***번길 ** ~ ** 사이 사거리 모서리쪽에 주차 못하게 차단...,주차 차단봉 설치


In [7]:
# 결측치가 있다면 제거
df = df.dropna()
df.shape

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(3081, 6)

In [8]:
# 결측치를 확인
df.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
Name,0
Value.age,0
Value.sex,0
Value.dep_name,0
UpdatedContent,0
UpdatedTitle,0


## 문서 만들기
* 제목과 내용을 함께 사용

In [9]:
df["문서"] = df["UpdatedTitle"] + " " + df["UpdatedContent"]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## 벡터화

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)


## CountVectorizer

* analyzer : 단어, 문자 단위의 벡터화 방법 정의
* ngram_range : BOW 단위 수 (1, 3) 이라면 1개~3개까지 토큰을 묶어서 벡터화
* max_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 높은 용어(말뭉치 관련 불용어)는 제외 (기본값=1.0)
    * max_df = 0.90 : 문서의 90% 이상에 나타나는 단어 제외
    * max_df = 10 : 10개 이상의 문서에 나타나는 단어 제외
* min_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 낮은 용어는 제외합니다. 컷오프라고도 합니다.(기본값=1.0)
    * min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 제외
    * min_df = 10 : 문서에 10개 미만으로 나타나는 단어 제외
* stop_words : 불용어 정의
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [10]:
# 단어들의 출현 빈도(frequency)로 여러 문서들을 벡터화하기 위해 CountVectorizer 사용
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=["돋움", "경우", "또는"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### 참고: fit, transform, fit_transfrom의 차이점
- fit(): 원시 문서에 있는 모든 토큰의 어휘 사전을 배운다
- transform(): 문서를 문서 용어 매트릭스로 변환, transform 이후엔 매트릭스로 변환되어 숫자형태로 변경
- fit_transform(): 어휘 사전을 배우고 문서 용어 매트릭스를 반환, fit 다음에 변환이 오는 것과 동일하지만 더 효율적으로 구현

* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform

In [11]:
# fit_transform을 사용하여 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_cv = cv.fit_transform(df["문서"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# cv.vocabulary_ 를 봅니다.
# cv.vocabulary_

In [12]:
cv_cols = cv.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector를 확인
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인

pd.DataFrame(dtm_cv.toarray(), columns=cv_cols).sum().sort_values()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
힘이듭니다,1
사항입니다,1
사회활동을,1
삭막한,1
산림이,1
...,...
원미구,1565
불법주차,2010
신고입니다,2042
경기도,2467


## 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

* API documentation: https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [16]:
# 정답인 '분류'의 유일한 값을 확인하여 주제 수를 확인
df["분류"].value_counts()

KeyError: '분류'

In [14]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옴
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수(기본값=10)

from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 10
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
# LDA_model 에 dtm_cv 를 넣어 학습
LDA_model.fit(dtm_cv)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(random_state=42)

### pyLDAvis

In [2]:
!pip install --upgrade pyLDAvis scikit-learn
#!pip install pyLDAvis==3.2.2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
import pyLDAvis
import pyLDAvis.gensim_models
import pandas as pd

pyLDAvis.enable_notebook()

# 최신 scikit-learn에서 get_feature_names_out() 사용
feature_names = cv.get_feature_names_out()

# pyLDAvis의 gensim_models 모듈을 사용하여 시각화 준비
# dtm_cv: 문서-단어 행렬, LDA_model: LDA 모델
pyLDAvis_display = pyLDAvis.prepare(
    topic_term_dists=LDA_model.components_,
    doc_topic_dists=LDA_model.transform(dtm_cv),
    doc_lengths=dtm_cv.sum(axis=1).getA1(),
    vocab=feature_names,
    term_frequency=dtm_cv.sum(axis=0).getA1()
)

# 결과 표시
pyLDAvis_display

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.319837 -0.008738       1        1  17.889895
6      0.290030 -0.008165       2        1  16.717718
7      0.121936  0.041436       3        1  15.590822
8     -0.102221  0.039229       4        1   9.619107
1     -0.053184  0.037873       5        1   8.391106
9     -0.099752 -0.264226       6        1   7.782863
3     -0.112969  0.059164       7        1   6.854390
2     -0.080036  0.015959       8        1   6.671782
5     -0.155624  0.038546       9        1   5.544018
4     -0.128017  0.048924      10        1   4.938300, topic_info=        Term          Freq         Total Category  logprob  loglift
11119    장애인  1.582976e+07  1.582976e+07  Default  30.0000  30.0000
7840   신고입니다  2.271805e+07  2.271805e+07  Default  29.0000  29.0000
11428   전용구역  1.405764e+07  1.405764e+07  Default  28.0000  28.0000
6221    불법주차  2.236251e+07  2.236251e+07  Default  27.0000  27.0000
13481   친환경차  8.916812e+06  8.916812e+06  Default  26.0000  26.0000
...      ...           ...           ...      ...      ...      ...
6844      상동  3.459158e+04  4.805391e+06  Topic10   2.3943   7.2191
2358      너무  2.570260e+04  8.458033e+05  Topic10   2.0973   8.6593
2752      다른  2.240768e+04  3.397809e+05  Topic10   1.9601   9.4341
7286     소사구  3.096676e+04  5.067576e+06  Topic10   2.2836   7.0553
10762     있는  2.341415e+04  8.530858e+05  Topic10   2.0040   8.5575

[637 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
9          1  0.012167   brt
9          2  0.011369   brt
9          3  0.010602   brt
9          4  0.006547   brt
9          5  0.005706   brt
...      ...       ...   ...
15137      6  0.015544  훼손방치
15137      7  0.013699  훼손방치
15137      8  0.813447  훼손방치
15137      9  0.011072  훼손방치
15137     10  0.009883  훼손방치

[4770 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 8, 9, 2, 10, 4, 3, 6, 5])

## TF-IDF(Term Frequency - Inverse Document Frequency)

## TfidfVectorizer

TF-IDF 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어(낮은 구별력)의 경우 가중치를 축소하는 방법

매개변수
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다.
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer


In [17]:
# TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩하여 벡터화하기 위해 TfidfVectorizer를 사용

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"])
tfidf

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


TfidfVectorizer(stop_words=['돋움', '경우', '또는', '있습니다', '있는', '합니다'])

In [18]:
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성
dtm_tfidf = tfidf.fit_transform(df["문서"])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
# tfidf.vocabulary_
cols_tfidf = tfidf.get_feature_names_out()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
# dtm_tf를 axis=0(수직 방향으로) 기준으로 합계를 낸 dist 변수를 생성
# dist 변수를 vocabulary_ 순으로 정렬하여 비율을 확인
import numpy as np
dist = np.sum(dtm_tfidf, axis=0)
pd.DataFrame(dist, columns=cols_tfidf).T.sort_values(by=0).tail(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
원미구,153.751424
부천시,170.507163
경기도,171.608916
안전신문고,175.464416
충전구역,188.434070
친환경차,192.966454
전용구역,254.872561
장애인,263.224024
신고입니다,336.537353
불법주차,337.978490


In [23]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 가중치를 적용한 vector를 확인
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인
pd.DataFrame(dtm_tfidf.toarray(), columns=cols_tfidf)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,aa,ai,allalr,alrno,amp,ars로만,a동,beom,bound,brt,...,힘들어서,힘들어요,힘들지,힘듧니다,힘듬,힘듭니,힘듭니다,힘써,힘써주세요,힘이듭니다
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3077,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3078,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 코사인 유사도
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [24]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(dtm_tfidf[0] , dtm_tfidf)
result_list = similarity_simple_pair.tolist()[0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
df["유사도"] = result_list
df[["Value.dep_name", "UpdatedTitle", "유사도"]].sort_values(by="유사도", ascending=False).head(10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Value.dep_name,UpdatedTitle,유사도
0,건설안전과,안녕하세요 여기는. 오정동 우체국 옆 상가 주차장 입,1.000000
2463,기후에너지과,친환경차 충전구역 불법주차 신고입니다.**너****,0.184303
819,건설안전과,오정동 도로 파임* 안전신문고 신고파일(사진·동영상,0.167439
1303,건설안전과,촬,0.140731
212,건설안전과,도로시설물불량,0.130824
1644,장애인복지과,장애인 전용구역 불법주차 신고입니다.* 안전신문고,0.127487
2476,장애인복지과,장애인 전용구역 불법주차 신고입니다.* 안전신문고,0.127487
2087,장애인복지과,장애인 전용구역 불법주차 신고입니다.* 안전신문고,0.127487
2688,장애인복지과,장애인 전용구역 불법주차 신고입니다.* 안전신문고,0.127487
2957,장애인복지과,장애인 전용구역 불법주차 신고입니다.* 안전신문고,0.127487
